# Building the model

The purpose of this notebook is to build the model that will eventually be used as the model behind the PL prediction for the app, and building the script to rebuild the model when new player data is introduced weekly

## Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score
from scikeras.wrappers import KerasRegressor
import tensorflow as tf
from sklearn.model_selection import GridSearchCV

## Loading the dataset

In [2]:
combined = pd.read_csv("../final_combined_dataframe.csv")

In [3]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1927 entries, 0 to 1926
Data columns (total 89 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Unnamed: 0                          1927 non-null   int64  
 1   match_id                            1927 non-null   object 
 2   goals                               1927 non-null   float64
 3   assists                             1927 non-null   float64
 4   non_penalty_goals                   1927 non-null   float64
 5   penalties_scored                    1927 non-null   float64
 6   penalties_attempted                 1927 non-null   float64
 7   yellow_cards                        1927 non-null   float64
 8   red_cards                           1927 non-null   float64
 9   expected_goals                      1927 non-null   float64
 10  non_penalty_expected_goals          1927 non-null   float64
 11  expected_assisted_goals             1927 no

In [4]:
combined.head()

,Unnamed: 0,match_id,goals,assists,non_penalty_goals,penalties_scored,penalties_attempted,yellow_cards,red_cards,expected_goals,...,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
0,0,m-00381,0.907776,0.396142,0.738889,0.168887,0.236302,0.093022,-0.048172,0.776569,...,6,4,2,5,11,8,2,1,0,0
1,1,m-00382,0.155008,0.133838,0.123306,0.031702,0.039064,0.131687,-0.000329,0.200098,...,4,1,7,4,11,9,1,1,0,0
2,2,m-00383,-0.039926,-0.056540,-0.137227,0.097301,0.109960,0.066208,0.011733,-0.109844,...,6,9,5,5,9,11,1,2,0,0
3,3,m-00384,0.133878,-0.008924,0.122145,0.011733,0.011733,0.185723,-0.023049,0.212802,...,1,4,2,5,9,8,2,1,0,0
4,4,m-00385,0.401345,0.138483,0.411797,-0.010452,-0.001508,0.071065,-0.018950,0.419927,...,2,5,3,5,11,12,2,2,0,0


## Column Lists

In [5]:
output_columns = [
	"home_goals", "away_goals", "home_shots", "away_shots", "home_shots_on_target", "away_shots_on_target",
	"home_corners", "away_corners", "home_fouls", "away_fouls", "home_yellow_cards", "away_yellow_cards",
	"home_red_cards", "away_red_cards"
]

player_stats_columns = [
	"player_id", "minutes_played","ninetys","goals","assists","non_penalty_goals","penalties_scored","penalties_attempted","yellow_cards","red_cards","expected_goals",
	"non_penalty_expected_goals","expected_assisted_goals","progressive_carries","progressive_passes","progressive_passes_received","total_passing_distance","total_progressive_passing_distance","short_passes_completed","short_passes_attempted","medium_passes_completed","medium_passes_attempted",
	"long_passes_completed","long_passes_attempted","expected_assists","key_passes","passes_into_final_third","passes_into_penalty_area","crosses_into_penalty_area","shots","shots_on_target","average_shot_distance","shots_from_free_kicks",
	"shots_from_penalties","touches","touches_in_defensive_penalty_area","touches_in_defensive_third","touches_in_middle_third","touches_in_attacking_third","touches_in_attacking_penalty_area","live_ball_touches","take_ons_attempted","take_ons_succeeded","times_tackled_during_take_on",
	"carries","total_carrying_distance","progressive_carrying_distance","carries_into_final_third","carries_into_penalty_area","miscontrols","dispossessed","passes_received","tackles","tackles_won","defensive_third_tackles",
	"middle_third_tackles","attacking_third_tackles","dribblers_tackled","dribbler_tackles_attempted","shots_blocked","passes_blocked","interceptions","clearances","errors_leading_to_shot","goals_against","shots_on_target_against","saves","clean_sheets","penalties_faced","penalties_allowed","penalties_saved","penalties_missed"
]
pure_stats_columns = [
	"minutes_played","goals","assists","non_penalty_goals","penalties_scored","penalties_attempted","yellow_cards","red_cards","expected_goals",
	"non_penalty_expected_goals","expected_assisted_goals","progressive_carries","progressive_passes","progressive_passes_received","total_passing_distance","total_progressive_passing_distance","short_passes_completed","short_passes_attempted","medium_passes_completed","medium_passes_attempted",
	"long_passes_completed","long_passes_attempted","expected_assists","key_passes","passes_into_final_third","passes_into_penalty_area","crosses_into_penalty_area","shots","shots_on_target","average_shot_distance","shots_from_free_kicks",
	"shots_from_penalties","touches","touches_in_defensive_penalty_area","touches_in_defensive_third","touches_in_middle_third","touches_in_attacking_third","touches_in_attacking_penalty_area","live_ball_touches","take_ons_attempted","take_ons_succeeded","times_tackled_during_take_on",
	"carries","total_carrying_distance","progressive_carrying_distance","carries_into_final_third","carries_into_penalty_area","miscontrols","dispossessed","passes_received","tackles","tackles_won","defensive_third_tackles",
	"middle_third_tackles","attacking_third_tackles","dribblers_tackled","dribbler_tackles_attempted","shots_blocked","passes_blocked","interceptions","clearances","errors_leading_to_shot","goals_against","shots_on_target_against","saves","clean_sheets","penalties_faced","penalties_allowed","penalties_saved","penalties_missed"
]
team_stats_columns = [
	"team_id", "goals","assists","non_penalty_goals","penalties_scored","penalties_attempted","yellow_cards","red_cards","expected_goals",
	"non_penalty_expected_goals","expected_assisted_goals","progressive_carries","progressive_passes","progressive_passes_received","total_passing_distance","total_progressive_passing_distance","short_passes_completed","short_passes_attempted","medium_passes_completed","medium_passes_attempted",
	"long_passes_completed","long_passes_attempted","expected_assists","key_passes","passes_into_final_third","passes_into_penalty_area","crosses_into_penalty_area","shots","shots_on_target","average_shot_distance","shots_from_free_kicks",
	"shots_from_penalties","touches","touches_in_defensive_penalty_area","touches_in_defensive_third","touches_in_middle_third","touches_in_attacking_third","touches_in_attacking_penalty_area","live_ball_touches","take_ons_attempted","take_ons_succeeded","times_tackled_during_take_on",
	"carries","total_carrying_distance","progressive_carrying_distance","carries_into_final_third","carries_into_penalty_area","miscontrols","dispossessed","passes_received","tackles","tackles_won","defensive_third_tackles",
	"middle_third_tackles","attacking_third_tackles","dribblers_tackled","dribbler_tackles_attempted","shots_blocked","passes_blocked","interceptions","clearances","errors_leading_to_shot","goals_against","shots_on_target_against","saves","clean_sheets","penalties_faced","penalties_allowed","penalties_saved","penalties_missed"
]
pure_stats_columns_no_minutes = [
	"goals","assists","non_penalty_goals","penalties_scored","penalties_attempted","yellow_cards","red_cards","expected_goals",
	"non_penalty_expected_goals","expected_assisted_goals","progressive_carries","progressive_passes","progressive_passes_received","total_passing_distance","total_progressive_passing_distance","short_passes_completed","short_passes_attempted","medium_passes_completed","medium_passes_attempted",
	"long_passes_completed","long_passes_attempted","expected_assists","key_passes","passes_into_final_third","passes_into_penalty_area","crosses_into_penalty_area","shots","shots_on_target","average_shot_distance","shots_from_free_kicks",
	"shots_from_penalties","touches","touches_in_defensive_penalty_area","touches_in_defensive_third","touches_in_middle_third","touches_in_attacking_third","touches_in_attacking_penalty_area","live_ball_touches","take_ons_attempted","take_ons_succeeded","times_tackled_during_take_on",
	"carries","total_carrying_distance","progressive_carrying_distance","carries_into_final_third","carries_into_penalty_area","miscontrols","dispossessed","passes_received","tackles","tackles_won","defensive_third_tackles",
	"middle_third_tackles","attacking_third_tackles","dribblers_tackled","dribbler_tackles_attempted","shots_blocked","passes_blocked","interceptions","clearances","errors_leading_to_shot","goals_against","shots_on_target_against","saves","clean_sheets","penalties_faced","penalties_allowed","penalties_saved","penalties_missed"
]

## Scaling

In [6]:
scaler = StandardScaler(copy=True)

In [7]:
combined_standardized = scaler.fit_transform(combined[pure_stats_columns_no_minutes])

In [8]:
evr = {}
n_components = {}
feature_to_pc = {}

for n in range(1,26):
	pca = PCA(n_components = n, random_state=938)
	pca.fit(combined_standardized)
	feature_to_pc_map = pd.DataFrame(pca.components_, columns=pure_stats_columns_no_minutes)
	components = pca.transform(combined_standardized)
	components_df = pd.DataFrame(data=components[:, [p for p in range(n)]], columns=pca.get_feature_names_out(), )
	
	evr[n] = sum(pca.explained_variance_ratio_)
	n_components[n] = components_df
	feature_to_pc[n] = feature_to_pc_map

pd.DataFrame(data=evr, index=["explained_variance_ratio"]).T
feature_to_pc[15].to_csv("../feature_to_15_pcs.csv")


Check the effect of different numbers if PCs on the outcome of the NN, testing n=2, 5 and 10. Use each n components to train and test the neural network, and compare the performance of each one

## Number of Features

In [9]:
INPUT_SIZE = 69
OUTPUT_SIZE = 14
ROWS = 1927

### n = 2

Train test split of data

In [10]:
n=2
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

Model definition

In [11]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(32, activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500
 1/31 [..............................] - ETA: 4s - loss: 55.1261 - accuracy: 0.0200

2024-01-10 23:54:32.161264: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


31/31 [==============================] - 0s 417us/step - loss: 29.1510 - accuracy: 0.4238
Epoch 2/500
31/31 [==============================] - 0s 289us/step - loss: 16.9076 - accuracy: 0.4666
Epoch 3/500
31/31 [==============================] - 0s 293us/step - loss: 13.7981 - accuracy: 0.4692
Epoch 4/500
31/31 [==============================] - 0s 320us/step - loss: 10.9975 - accuracy: 0.4672
Epoch 5/500
31/31 [==============================] - 0s 305us/step - loss: 9.3855 - accuracy: 0.4685
Epoch 6/500
31/31 [==============================] - 0s 296us/step - loss: 8.9588 - accuracy: 0.4679
Epoch 7/500
31/31 [==============================] - 0s 301us/step - loss: 9.0663 - accuracy: 0.4711
Epoch 8/500
31/31 [==============================] - 0s 301us/step - loss: 8.5792 - accuracy: 0.4705
Epoch 9/500
31/31 [==============================] - 0s 289us/step - loss: 8.4374 - accuracy: 0.4744
Epoch 10/500
31/31 [==============================] - 0s 297us/step - loss: 8.6485 - accuracy: 0.47

In [12]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

13/13 [==============================] - 0s 288us/step
[1.5289333e+00 1.3014370e+00 1.3063078e+01 1.1743839e+01 4.6338062e+00
 4.1374846e+00 5.2314868e+00 4.6817131e+00 1.0428206e+01 1.0873027e+01
 1.4668657e+00 1.8280799e+00 1.5001114e-02 9.1019301e-03]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1489,1,5,13,13,5,7,7,3,4,6,0,1,0,0
643,0,0,21,8,4,1,11,7,11,9,4,3,0,0
1263,1,2,9,17,3,7,4,11,8,12,0,0,0,0
848,1,3,19,15,6,6,5,4,9,10,0,0,0,0
95,0,4,7,24,1,8,4,4,14,10,4,2,0,0


### n = 5

In [13]:
n=5
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

In [14]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(32, activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500
31/31 [==============================] - 0s 392us/step - loss: 26.5683 - accuracy: 0.4640
Epoch 2/500
31/31 [==============================] - 0s 370us/step - loss: 15.3612 - accuracy: 0.4698
Epoch 3/500
31/31 [==============================] - 0s 328us/step - loss: 12.7946 - accuracy: 0.4646
Epoch 4/500
31/31 [==============================] - 0s 305us/step - loss: 11.2298 - accuracy: 0.4705
Epoch 5/500
31/31 [==============================] - 0s 302us/step - loss: 10.0638 - accuracy: 0.4711
Epoch 6/500
31/31 [==============================] - 0s 305us/step - loss: 9.2993 - accuracy: 0.4737
Epoch 7/500
31/31 [==============================] - 0s 301us/step - loss: 8.9228 - accuracy: 0.4731
Epoch 8/500
31/31 [==============================] - 0s 302us/step - loss: 9.1725 - accuracy: 0.4711
Epoch 9/500
31/31 [==============================] - 0s 303us/step - loss: 8.7850 - accuracy: 0.4731
Epoch 10/500
31/31 [==============================] - 0s 319us/step - loss: 8.7895 - a

In [15]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

13/13 [==============================] - 0s 479us/step
[ 1.4912498   1.4929992  13.207611   11.625853    4.728223    4.2126856
  5.0069256   4.6086802  10.116126   10.656878    1.2975084   1.7179339
 -0.05096611  0.07372776]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1489,1,5,13,13,5,7,7,3,4,6,0,1,0,0
643,0,0,21,8,4,1,11,7,11,9,4,3,0,0
1263,1,2,9,17,3,7,4,11,8,12,0,0,0,0
848,1,3,19,15,6,6,5,4,9,10,0,0,0,0
95,0,4,7,24,1,8,4,4,14,10,4,2,0,0


### n = 10

In [16]:
n=10
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

In [17]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(32, activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500
31/31 [==============================] - 0s 374us/step - loss: 26.4214 - accuracy: 0.3660
Epoch 2/500
31/31 [==============================] - 0s 327us/step - loss: 14.3190 - accuracy: 0.4672
Epoch 3/500
31/31 [==============================] - 0s 304us/step - loss: 11.9710 - accuracy: 0.4666
Epoch 4/500
31/31 [==============================] - 0s 299us/step - loss: 10.6242 - accuracy: 0.4731
Epoch 5/500
31/31 [==============================] - 0s 289us/step - loss: 9.8525 - accuracy: 0.4750
Epoch 6/500
31/31 [==============================] - 0s 298us/step - loss: 9.3703 - accuracy: 0.4731
Epoch 7/500
31/31 [==============================] - 0s 291us/step - loss: 9.0964 - accuracy: 0.4737
Epoch 8/500
31/31 [==============================] - 0s 293us/step - loss: 9.0667 - accuracy: 0.4692
Epoch 9/500
31/31 [==============================] - 0s 327us/step - loss: 8.8365 - accuracy: 0.4640
Epoch 10/500
31/31 [==============================] - 0s 334us/step - loss: 8.6659 - ac

In [18]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

13/13 [==============================] - 0s 364us/step
[ 1.79302990e+00  8.53443742e-01  1.71868286e+01  8.92531872e+00
  5.58224392e+00  3.38966346e+00  6.59222412e+00  3.81947494e+00
  1.19019985e+01  1.22370081e+01  1.73466265e+00  2.02354765e+00
 -2.59541988e-01  2.36746855e-03]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1489,1,5,13,13,5,7,7,3,4,6,0,1,0,0
643,0,0,21,8,4,1,11,7,11,9,4,3,0,0
1263,1,2,9,17,3,7,4,11,8,12,0,0,0,0
848,1,3,19,15,6,6,5,4,9,10,0,0,0,0
95,0,4,7,24,1,8,4,4,14,10,4,2,0,0


### n=15

In [19]:
n=15
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

In [20]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(32, activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500
31/31 [==============================] - 0s 413us/step - loss: 26.4100 - accuracy: 0.4004
Epoch 2/500
31/31 [==============================] - 0s 318us/step - loss: 14.0804 - accuracy: 0.4646
Epoch 3/500
31/31 [==============================] - 0s 334us/step - loss: 11.6436 - accuracy: 0.4685
Epoch 4/500
31/31 [==============================] - 0s 301us/step - loss: 10.5152 - accuracy: 0.4705
Epoch 5/500
31/31 [==============================] - 0s 298us/step - loss: 9.7834 - accuracy: 0.4672
Epoch 6/500
31/31 [==============================] - 0s 298us/step - loss: 9.2447 - accuracy: 0.4679
Epoch 7/500
31/31 [==============================] - 0s 299us/step - loss: 9.0931 - accuracy: 0.4679
Epoch 8/500
31/31 [==============================] - 0s 298us/step - loss: 8.8632 - accuracy: 0.4672
Epoch 9/500
31/31 [==============================] - 0s 294us/step - loss: 8.7845 - accuracy: 0.4692
Epoch 10/500
31/31 [==============================] - 0s 307us/step - loss: 8.7403 - ac

In [21]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

13/13 [==============================] - 0s 327us/step
[ 1.24882     1.5427647  13.501266   11.082408    4.799042    4.4722743
  5.0357904   3.3096647  10.696568   11.801494    1.5503445   1.8767405
  0.50267124 -0.1643985 ]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1489,1,5,13,13,5,7,7,3,4,6,0,1,0,0
643,0,0,21,8,4,1,11,7,11,9,4,3,0,0
1263,1,2,9,17,3,7,4,11,8,12,0,0,0,0
848,1,3,19,15,6,6,5,4,9,10,0,0,0,0
95,0,4,7,24,1,8,4,4,14,10,4,2,0,0


### n=20

In [22]:
n=20
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

In [23]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(32, activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500
31/31 [==============================] - 0s 1ms/step - loss: 27.5639 - accuracy: 0.3835
Epoch 2/500
31/31 [==============================] - 0s 603us/step - loss: 14.2566 - accuracy: 0.4666
Epoch 3/500
31/31 [==============================] - 0s 336us/step - loss: 11.6875 - accuracy: 0.4692
Epoch 4/500
31/31 [==============================] - 0s 300us/step - loss: 10.5035 - accuracy: 0.4692
Epoch 5/500
31/31 [==============================] - 0s 300us/step - loss: 9.7979 - accuracy: 0.4731
Epoch 6/500
31/31 [==============================] - 0s 298us/step - loss: 9.3586 - accuracy: 0.4731
Epoch 7/500
31/31 [==============================] - 0s 291us/step - loss: 9.1087 - accuracy: 0.4705
Epoch 8/500
31/31 [==============================] - 0s 285us/step - loss: 8.8434 - accuracy: 0.4711
Epoch 9/500
31/31 [==============================] - 0s 265us/step - loss: 8.8823 - accuracy: 0.4724
Epoch 10/500
31/31 [==============================] - 0s 307us/step - loss: 8.6864 - accu

In [24]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

13/13 [==============================] - 0s 308us/step
[ 1.1289647   1.1567096  13.784657   12.268388    4.6290603   4.0934925
  5.2028384   4.427797   12.006419   11.495908    1.606045    2.0914881
  0.18874729 -0.10198753]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1489,1,5,13,13,5,7,7,3,4,6,0,1,0,0
643,0,0,21,8,4,1,11,7,11,9,4,3,0,0
1263,1,2,9,17,3,7,4,11,8,12,0,0,0,0
848,1,3,19,15,6,6,5,4,9,10,0,0,0,0
95,0,4,7,24,1,8,4,4,14,10,4,2,0,0


The above experiments show us the accuracy of each model increases with an increased number of features up to a point, it seem somewhere between 15 and 20 features, where the accuracy on the training data peaks. I will use 15 principal components going forward for the investigations, as this seems to yield the highest accuracy of those tested here

In terms of predicting exact values for the match facts, I am not expecting exact matches between model and the actual results, just hoping for a close approximation, or at the very least, the correct side with the higher value, i.e. home team scores more goals than the away team is correct

## 15 PC model build

#### Splitting data

In [25]:
X = combined[pure_stats_columns_no_minutes]
y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=543)

#### Scaling X and y

In [26]:
X_scaler = StandardScaler(copy=True).fit(X_train)
y_scaler = StandardScaler(copy=True).fit(y_train)

X_train = X_scaler.transform(X_train)
# y_train = y_scaler.transform(y_train)

X_test = X_scaler.transform(X_test)
# y_test = y_scaler.transform(y_test)

#### Applying PCA

In [27]:
n=15

pca = PCA(n_components = n, random_state=576)
pca.fit(X_train)

X_train = pca.transform(X_train)
X_test = pca.transform(X_test)

The below table shows the breakdown of each principle component in terms of the proportion of the value from each of the table columns used to make it up

In [28]:
df = pd.DataFrame(pca.components_, columns=pure_stats_columns_no_minutes)
df.index.name = "principle_component_number"
df

,goals,assists,non_penalty_goals,penalties_scored,penalties_attempted,yellow_cards,red_cards,expected_goals,non_penalty_expected_goals,expected_assisted_goals,...,clearances,errors_leading_to_shot,goals_against,shots_on_target_against,saves,clean_sheets,penalties_faced,penalties_allowed,penalties_saved,penalties_missed
principle_component_number,,,,,,,,,,,,,,,,,,,,,
0,-0.114827,-0.129052,-0.116702,-0.067303,-0.068918,-0.133266,-0.078929,-0.119126,-0.120603,-0.134648,...,-0.115752,-0.110908,-0.055217,-0.056287,-0.056471,-0.050583,-0.051109,-0.050263,-0.037204,-0.027362
1,-0.076207,-0.074874,-0.096449,-0.099214,-0.100669,0.002926,0.017051,-0.102086,-0.096433,-0.071295,...,0.037691,0.155209,0.299717,0.302302,0.301513,0.281103,0.297094,0.282476,0.247426,0.170881
2,0.129684,0.094786,0.202589,0.201222,0.203529,-0.091492,-0.078626,0.197089,0.184715,0.083927,...,-0.199649,-0.070431,0.147101,0.148932,0.149239,0.149440,0.149437,0.141045,0.125463,0.091498
3,0.012339,-0.056923,0.047812,0.517905,0.509710,0.044948,-0.002697,0.115383,0.042584,-0.049077,...,0.120272,0.070535,-0.014020,-0.015480,-0.015441,-0.007412,-0.033999,-0.026027,-0.035921,-0.050466
4,0.393711,-0.057483,0.078656,0.006452,-0.016994,0.072440,-0.070190,0.100952,0.114822,-0.048546,...,0.052095,0.001662,-0.029240,-0.029546,-0.031542,-0.026292,0.011700,0.004464,0.043552,-0.007951
5,0.065291,0.118182,-0.177586,0.105684,0.110093,-0.096678,-0.108414,-0.176219,-0.214140,0.138487,...,-0.280313,-0.106535,0.047202,0.042639,0.040278,0.033782,0.012787,0.051564,0.025091,-0.300169
6,0.027925,0.089704,-0.001078,-0.024269,-0.035036,-0.091844,0.046270,-0.024446,-0.021329,0.053927,...,-0.046283,-0.007501,-0.111597,-0.071465,-0.052535,0.029501,-0.072881,-0.161501,-0.174140,0.824104
7,-0.060446,0.059229,0.035370,-0.041391,-0.056498,-0.041428,-0.863648,0.018545,0.030034,0.034526,...,0.116808,0.017207,-0.017978,-0.015927,-0.013996,-0.006914,-0.022590,0.023737,-0.142707,-0.107514
8,-0.015947,-0.021876,-0.129169,0.147798,0.148286,0.112371,-0.392915,-0.066364,-0.098471,0.005349,...,0.048611,0.029535,0.014670,0.026517,0.031168,-0.018400,0.064582,0.028775,-0.000350,0.369304


#### Model Definition

In [29]:
INPUT_SIZE = len(X_train[0])
OUTPUT_SIZE = len(output_columns)

Define

In [30]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(15, activation='relu'),
	tf.keras.layers.Dense(12, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE, activation='relu')
])

Compile

In [31]:
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")

Fit

In [32]:
model.fit(X_train, y_train, epochs=500, batch_size=25);

Epoch 1/500
62/62 [==============================] - 0s 336us/step - loss: 33.8665 - accuracy: 0.4361
Epoch 2/500
62/62 [==============================] - 0s 270us/step - loss: 20.5366 - accuracy: 0.4744
Epoch 3/500
62/62 [==============================] - 0s 259us/step - loss: 18.2229 - accuracy: 0.4737
Epoch 4/500
62/62 [==============================] - 0s 265us/step - loss: 17.3387 - accuracy: 0.4757
Epoch 5/500
62/62 [==============================] - 0s 258us/step - loss: 16.9477 - accuracy: 0.4770
Epoch 6/500
62/62 [==============================] - 0s 256us/step - loss: 16.8210 - accuracy: 0.4757
Epoch 7/500
62/62 [==============================] - 0s 258us/step - loss: 16.7474 - accuracy: 0.4822
Epoch 8/500
62/62 [==============================] - 0s 259us/step - loss: 16.7602 - accuracy: 0.4789
Epoch 9/500
62/62 [==============================] - 0s 255us/step - loss: 16.6621 - accuracy: 0.4770
Epoch 10/500
62/62 [==============================] - 0s 253us/step - loss: 16.589

#### Testing

In [33]:
y_hat = model.predict(X_test)

13/13 [==============================] - 0s 307us/step


In [34]:
total_shots = 0
total_shots_over = 0
total_shots_under = 0

total_shots_on_target = 0
total_shots_on_target_over = 0
total_shots_on_target_under = 0

total_booking_points = 0
total_booking_points_over = 0
total_booking_points_under = 0

correct_score = 0

winner = 0

total_corners = 0
total_corners_over = 0
total_corners_under = 0

total_fouls = 0
total_fouls_over = 0
total_fouls_under = 0

goals_over = 0
goals_under = 0

all_under = 0
all_over = 0

total_tested  = len(y_hat)

In [35]:
for idx, y in enumerate(y_hat):
	home_goals_hat, away_goals_hat, home_shots_hat, away_shots_hat, home_shots_on_target_hat, away_shots_on_target_hat, home_corners_hat, away_corners_hat, home_fouls_hat, away_fouls_hat, home_yellow_cards_hat, away_yellow_cards_hat, home_red_cards_hat, away_red_cards_hat = y
	home_goals, away_goals, home_shots, away_shots, home_shots_on_target, away_shots_on_target, home_corners, away_corners, home_fouls, away_fouls, home_yellow_cards, away_yellow_cards, home_red_cards, away_red_cards = y_test.iloc[idx].tolist()
	
	total_shots += 1 if np.floor(home_shots_hat)+np.floor(away_shots_hat) == home_shots+away_shots else 0
	total_shots_over += 1 if np.floor(home_shots_hat)+np.floor(away_shots_hat) > home_shots+away_shots else 0
	total_shots_under += 1 if np.floor(home_shots_hat)+np.floor(away_shots_hat) < home_shots+away_shots else 0
	
	total_shots_on_target += 1 if np.floor(home_shots_on_target_hat)+np.floor(away_shots_on_target_hat) == home_shots_on_target+away_shots_on_target else 0
	total_shots_on_target_over += 1 if np.floor(home_shots_on_target_hat)+np.floor(away_shots_on_target_hat) > home_shots_on_target+away_shots_on_target else 0
	total_shots_on_target_under += 1 if np.floor(home_shots_on_target_hat)+np.floor(away_shots_on_target_hat) < home_shots_on_target+away_shots_on_target else 0

	total_booking_points += 1 if (np.floor(home_yellow_cards_hat)+np.floor(away_yellow_cards_hat))*10+(np.floor(home_red_cards_hat)+np.floor(away_red_cards_hat))*25 == (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 else 0
	total_booking_points_over += 1 if (np.floor(home_yellow_cards_hat)+np.floor(away_yellow_cards_hat))*10+(np.floor(home_red_cards_hat)+np.floor(away_red_cards_hat))*25 > (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 else 0
	total_booking_points_under += 1 if (np.floor(home_yellow_cards_hat)+np.floor(away_yellow_cards_hat))*10+(np.floor(home_red_cards_hat)+np.floor(away_red_cards_hat))*25 < (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 else 0

	correct_score += 1 if np.floor(home_goals_hat) == home_goals and np.floor(away_goals_hat) == away_goals else 0
	
	winner += 1 if (home_goals_hat > away_goals_hat and home_goals > away_goals) or (home_goals_hat < away_goals_hat and home_goals < away_goals) else 0

	total_fouls += 1 if np.floor(home_fouls_hat) + np.floor(away_fouls_hat) == home_fouls + away_fouls else 0
	total_fouls_over += 1 if np.floor(home_fouls_hat) + np.floor(away_fouls_hat) > home_fouls+ away_fouls else 0
	total_fouls_under += 1 if np.floor(home_fouls_hat) + np.floor(away_fouls_hat) < home_fouls+ away_fouls else 0

	total_corners += 1 if np.floor(home_corners_hat) + np.floor(away_corners_hat) == home_corners + away_corners else 0
	total_corners_over += 1 if np.floor(home_corners_hat) + np.floor(away_corners_hat) > home_corners + away_corners else 0
	total_corners_under += 1 if np.floor(home_corners_hat) + np.floor(away_corners_hat) < home_corners + away_corners else 0

	goals_over += 1 if np.floor(home_goals_hat) + np.floor(away_goals_hat) > home_goals + away_goals else 0
	goals_under += 1 if np.floor(home_goals_hat) + np.floor(away_goals_hat) < home_goals + away_goals else 0

	all_under += 1 if np.floor(home_shots_hat)+np.floor(away_shots_hat) < home_shots+away_shots and np.floor(home_shots_on_target_hat)+np.floor(away_shots_on_target_hat) < home_shots_on_target+away_shots_on_target and (np.floor(home_yellow_cards_hat)+np.floor(away_yellow_cards_hat))*10+(np.floor(home_red_cards_hat)+np.floor(away_red_cards_hat))*25 < (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 and np.floor(home_corners_hat) + np.floor(away_corners_hat) < home_corners + away_corners and np.floor(home_corners_hat) + np.floor(away_corners_hat) < home_corners + away_corners and np.floor(home_goals_hat) + np.floor(away_goals_hat) < home_goals + away_goals else 0
	all_over += 1 if np.floor(home_shots_hat)+np.floor(away_shots_hat) > home_shots+away_shots and np.floor(home_shots_on_target_hat)+np.floor(away_shots_on_target_hat) > home_shots_on_target+away_shots_on_target and (np.floor(home_yellow_cards_hat)+np.floor(away_yellow_cards_hat))*10+(np.floor(home_red_cards_hat)+np.floor(away_red_cards_hat))*25 > (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 and np.floor(home_corners_hat) + np.floor(away_corners_hat) > home_corners + away_corners and np.floor(home_goals_hat) + np.floor(away_goals_hat) > home_goals + away_goals else 0

In [36]:
print("total_shots: "+str(total_shots/total_tested))
print("total_shots_over: "+str(total_shots_over/total_tested))
print("total_shots_under: "+str(total_shots_under/total_tested))
print("\n")
print("total_shots_on_target: "+str(total_shots_on_target/total_tested))
print("total_shots_on_target_over: "+str(total_shots_on_target_over/total_tested))
print("total_shots_on_target_under: "+str(total_shots_on_target_under/total_tested))
print("\n")
print("total_booking_points: "+str(total_booking_points/total_tested))
print("total_booking_points_over: "+str(total_booking_points_over/total_tested))
print("total_booking_points_under: "+str(total_booking_points_under/total_tested))
print("\n")
print("correct_score: "+str(correct_score/total_tested))
print("\n")
print("winner: "+str(winner/total_tested))
print("\n")
print("total_corners: "+str(total_corners/total_tested))
print("total_corners_over: "+str(total_corners_over/total_tested))
print("total_corners_under: "+str(total_corners_under/total_tested))
print("\n")
print("total_fouls: "+str(total_fouls/total_tested))
print("total_fouls_over: "+str(total_fouls_over/total_tested))
print("total_fouls_under: "+str(total_fouls_under/total_tested))
print("\n")
print("goals_over: "+str(goals_over/total_tested))
print("goals_under: "+str(goals_under/total_tested))
print("\n")
print("all_over: "+str(all_over/total_tested))
print("all_under: "+str(all_under/total_tested))

total_shots: 0.05958549222797927
total_shots_over: 0.42487046632124353
total_shots_under: 0.5155440414507773


total_shots_on_target: 0.12435233160621761
total_shots_on_target_over: 0.3290155440414508
total_shots_on_target_under: 0.5466321243523317


total_booking_points: 0.17098445595854922
total_booking_points_over: 0.18652849740932642
total_booking_points_under: 0.6424870466321243


correct_score: 0.11658031088082901


winner: 0.4067357512953368


total_corners: 0.11398963730569948
total_corners_over: 0.38341968911917096
total_corners_under: 0.5025906735751295


total_fouls: 0.07253886010362694
total_fouls_over: 0.3471502590673575
total_fouls_under: 0.5803108808290155


goals_over: 0.20984455958549222
goals_under: 0.5569948186528497


all_over: 0.0025906735751295338
all_under: 0.09585492227979274


### Tuning

Create a function to build and compile a model, with the arguments in it being the values to be update during tuning

In [37]:
def get_mlp_model(hidden_layer_one=13, dropout=0.2, learn_rate=0.01, n_h_layers=1):

	model = tf.keras.models.Sequential()

	# input
	model.add(tf.keras.layers.Dense(15, activation="relu", input_dim=15))

	for i in range(n_h_layers):
		model.add(tf.keras.layers.Dense(hidden_layer_one, activation="relu"))

	# dropout layer to remove redundant nodes
	model.add(tf.keras.layers.Dropout(dropout))
	
	# output
	model.add(tf.keras.layers.Dense(14, activation="relu"))

	model.compile(
		optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=learn_rate),
		loss="mse",
		metrics=["accuracy"])

	return model

Build a model using the *get_mlp_model* function, letting tensorflow use this function when building the models

In [38]:
model = KerasRegressor(model=get_mlp_model, verbose=0, hidden_layer_one=10, learn_rate=0.01, dropout=0.05, n_h_layers=1);

Define the hyperparameter search space, and then generate the search object to be used later, by RandomizedSearchCV

In [40]:
# define a grid of the hyperparameter search space
hidden_layer_one = [10, 15, 30, 50, 100]
learn_rate = [1e-2, 1e-3, 1e-4]
dropout = [0.05, 0.1, 0.2, 0.3, 0.4]
batch_size = [4, 8, 16, 32, 64]
epochs = [10, 50, 100, 150, 200, 500]
n_h_layers = [1, 2, 3, 5]

# create a dictionary from the hyperparameter grid
grid = dict(
	hidden_layer_one=hidden_layer_one,
	learn_rate=learn_rate,
	dropout=dropout,
	batch_size=batch_size,
	epochs=epochs,
	n_h_layers=n_h_layers
)

Build a scoring function for the searcher, average the value of all the *under* stats and return this

In [41]:
def scoring(estimator, test_x: np.ndarray, test_y: pd.DataFrame) -> float:
	test_y = test_y.to_numpy()
	
	estimator.fit(test_x, test_y);
	y_hat = estimator.predict(test_x)
	
	average_under_rate = 0

	total_shots_under = 0
	total_shots_on_target_under = 0
	total_booking_points_under = 0
	total_corners_under = 0
	total_fouls_under = 0
	goals_under = 0

	total_tested = len(y_hat)
	
	for idx, y in enumerate(y_hat):

		home_goals_hat, away_goals_hat, home_shots_hat, away_shots_hat, home_shots_on_target_hat, away_shots_on_target_hat, home_corners_hat, away_corners_hat, home_fouls_hat, away_fouls_hat, home_yellow_cards_hat, away_yellow_cards_hat, home_red_cards_hat, away_red_cards_hat = y
		home_goals, away_goals, home_shots, away_shots, home_shots_on_target, away_shots_on_target, home_corners, away_corners, home_fouls, away_fouls, home_yellow_cards, away_yellow_cards, home_red_cards, away_red_cards = test_y[idx]

		total_shots_under += 1 if np.floor(home_shots_hat)+np.floor(away_shots_hat) < home_shots+away_shots else 0
		total_shots_on_target_under += 1 if np.floor(home_shots_on_target_hat)+np.floor(away_shots_on_target_hat) < home_shots_on_target+away_shots_on_target else 0
		total_booking_points_under += 1 if (np.floor(home_yellow_cards_hat)+np.floor(away_yellow_cards_hat))*10+(np.floor(home_red_cards_hat)+np.floor(away_red_cards_hat))*25 < (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 else 0
		total_corners_under += 1 if np.floor(home_corners_hat) + np.floor(away_corners_hat) < home_corners + away_corners else 0
		total_fouls_under += 1 if np.floor(home_fouls_hat) + np.floor(away_fouls_hat) < home_fouls+ away_fouls else 0
		goals_under += 1 if np.floor(home_goals_hat) + np.floor(away_goals_hat) < home_goals + away_goals else 0

		average_under_rate += ((total_shots_under/total_tested)+(total_shots_on_target_under/total_tested)+(total_booking_points_under/total_tested)+(total_fouls_under/total_tested)+(total_corners_under/total_tested)+(goals_under/total_tested))/6

	average_under_rate = average_under_rate/total_tested
	
	return average_under_rate

Generate the searcher object, used to iterate over the search parameters and determine the best hyper-parameter setup from the space we've created above

In [42]:
searcher = RandomSearchCV(estimator=model, n_jobs=-2, 
	param_grid=grid, scoring=scoring, verbose=4, cv=3)

searchResults = searcher.fit(X_train, y_train)

bestScore = searchResults.best_score_
bestParams = searchResults.best_params_
print("[INFO] best score is {:.2f} using {}".format(bestScore,bestParams))

NameError: name 'RandomSearchCV' is not defined

In [ ]:
tuned_model = searcher.best_estimator_.model
tuned_model.save("stats_regression_model.h5")

## Next v  steps

- Tune parameters and layers on the 15 PC model to increase the accuracy
- Create a method to define the accuracy on the test set; strict equality for each of the output values is not necessary as under/over options on the betting platform mean a close approximation of the outcome is all that is required. Success/accuracy measure based on the bet markets available for each game:
	- Total shots
	- Total booking points
	- Correct score
	- Outright
	- Total corners
	- Under/over goals
- Save the model
- Complete the notebook with all annotations and text

RandomSearchCV  
best score is 0.39 using {'n_h_layers': 5, 'learn_rate': 0.0001, 'hidden_layer_one': 15, 'epochs': 100, 'dropout': 0.2, 'batch_size': 64}  
best score is 0.40 using {'n_h_layers': 7, 'learn_rate': 0.0001, 'hidden_layer_one': 15, 'epochs': 1000, 'dropout': 0.75, 'batch_size': 16}  
best score is 0.49 using {'n_h_layers': 3, 'learn_rate': 0.0001, 'hidden_layer_one': 30, 'epochs': 10, 'dropout': 0.3, 'batch_size': 64} 

GridSearchCV  
best score is 0.47 using {'batch_size': 64, 'dropout': 0.4, 'epochs': 100, 'hidden_layer_one': 10, 'learn_rate': 0.0001, 'n_h_layers': 2}  
best score is 0.49100 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 10, 'hidden_layer_one': 10, 'learn_rate': 0.0001, 'n_h_layers': 3}  
best score is 0.49130 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 20, 'hidden_layer_one': 15, 'learn_rate': 1e-05, 'n_h_layers': 4}  
best score is 0.49132 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 20, 'hidden_layer_one': 15, 'learn_rate': 1e-06, 'n_h_layers': 4}  
best score is 0.49138 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 15, 'hidden_layer_one': 10, 'learn_rate': 0.0001, 'n_h_layers': 6}  
best score is 0.49138 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 15, 'hidden_layer_one': 10, 'learn_rate': 1e-05, 'n_h_layers': 6}  
best score is 0.49138 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 15, 'hidden_layer_one': 10, 'learn_rate': 1e-07, 'n_h_layers': 6}  
best score is 0.49138 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 15, 'hidden_layer_one': 10, 'learn_rate': 0.0001, 'n_h_layers': 4}  
best score is 0.49138 using {'batch_size': 32, 'dropout': 0.2, 'epochs': 12, 'hidden_layer_one': 10, 'learn_rate': 1e-05, 'n_h_layers': 4}  

- During
	- [CV 1/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=1;, score=0.459 total time=   4.3s
	- [CV 3/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=1;, score=0.462 total time=   3.2s
	- [CV 1/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=2;, score=0.430 total time=   3.5s
	- [CV 2/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=1;, score=0.458 total time=   5.3s
	- [CV 2/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=2;, score=0.412 total time=   3.8s
	- [CV 3/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=2;, score=0.451 total time=   3.5s
	- [CV 1/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=3;, score=0.447 total time=   3.8s
	- [CV 2/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=3;, score=0.413 total time=   4.8s
	- [CV 3/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=3;, score=0.415 total time=   4.9s

## Questions

- Scaling y reduces the accuracy of the model by a factor of 6, why?